# postgres python connections

In [20]:
!pip install psycopg2-binary

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/20/81/4940235d18747f865d47eb38b98f38acc24b39278b12e20a0fdd20e0a132/psycopg2_binary-2.9.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.9 MB/s eta 0:00:00a 0:00:01


In [55]:
import psycopg2

In [56]:
# Connect to your postgres DB
conn = psycopg2.connect("dbname=neo4jdb user=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
# cur.execute("SELECT * FROM my_data")

# Retrieve query results
# records = cur.fetchall()

## Queries

In [64]:
# query for employees table
createTableQueryEmp = \
"""CREATE TABLE IF NOT EXISTS employees (
    employee_id INT,
    full_name VARCHAR,
    job_title VARCHAR,
    manager_id INT,
    department_id INT,
    collaborator_id INT
);"""
# insert query for employees table
insertTableQueryEmp = \
"""
INSERT INTO 
    employees(employee_id, full_name, job_title, manager_id, department_id, collaborator_id)
VALUES
    (1, 'John Doe', 'CEO', 1, 1, NULL),
    (2, 'Jane Smith', 'CTO', 1, 1, NULL),
    (3, 'Jim Brown', 'Data Analyst', 2, 2, 4),
    (4, 'Judy White', 'Data Scientist', 2, 2, 3),
    (5, 'Joe Black', 'Sr. Software Eng.', 2, 3, 6),
    (6, 'Jennifer Green', 'Software Eng.', 5, 3, 5),
    (7, 'Jason Blue', 'Product Manager', 1, 4, 8),
    (8, 'Jane Orange', 'Product Designer', 7, 4, 7);
"""

# create query for departments
createTableQueryDept = \
"""CREATE TABLE IF NOT EXISTS departments (
    department_id INT,
    name VARCHAR
);"""
# insert query for departments table
insertTableQueryDept = \
"""
INSERT INTO 
    departments(department_id, name)
VALUES
(1, 'Management'),
(2, 'Data'),
(3, 'Engineering'),
(4, 'Products');
"""

### Employees table

In [65]:
# create employees table
cur.execute(createTableQueryEmp)

In [66]:
conn.commit()

In [67]:
cur.execute("SELECT * FROM employees;")
cur.fetchall()

[(1, 'John Doe', 'CEO', 1, 1, None),
 (2, 'Jane Smith', 'CTO', 1, 1, None),
 (3, 'Jim Brown', 'Data Analyst', 2, 2, 4),
 (4, 'Judy White', 'Data Scientist', 2, 2, 3),
 (5, 'Joe Black', 'Sr. Software Eng.', 2, 3, 6),
 (6, 'Jennifer Green', 'Software Eng.', 5, 3, 5),
 (7, 'Jason Blue', 'Product Manager', 1, 4, 8),
 (8, 'Jane Orange', 'Product Designer', 7, 4, 7)]

In [61]:
cur.execute(insertTableQueryEmp)

In [62]:
conn.commit()

In [68]:
cur.execute("SELECT * FROM employees;")
cur.fetchall()

[(1, 'John Doe', 'CEO', 1, 1, None),
 (2, 'Jane Smith', 'CTO', 1, 1, None),
 (3, 'Jim Brown', 'Data Analyst', 2, 2, 4),
 (4, 'Judy White', 'Data Scientist', 2, 2, 3),
 (5, 'Joe Black', 'Sr. Software Eng.', 2, 3, 6),
 (6, 'Jennifer Green', 'Software Eng.', 5, 3, 5),
 (7, 'Jason Blue', 'Product Manager', 1, 4, 8),
 (8, 'Jane Orange', 'Product Designer', 7, 4, 7)]

### Departments table

In [69]:
# create departments table
cur.execute(createTableQueryDept)

In [71]:
# insert into departments table
cur.execute(insertTableQueryDept)

In [74]:
# check insert data
cur.execute("SELECT * FROM departments;")
cur.fetchall()

[(1, 'Management'),
 (2, 'Data'),
 (3, 'Engineering'),
 (4, 'Products'),
 (1, 'Management'),
 (2, 'Data'),
 (3, 'Engineering'),
 (4, 'Products')]

In [75]:
# make changes persistent
conn.commit()

### fetch data into csv's

In [85]:
# fetch records from tables
cur.execute("SELECT * FROM employees;")
employees = cur.fetchall()
cur.execute("SELECT * FROM departments")
departments = cur.fetchall()

In [90]:
# create df and save to csv
empdf = pd.DataFrame(
    data=employees, 
    columns=["employee_id", "full_name", "job_title", "manager_id", "department_id", "collaborator_id"])
deptdf = pd.DataFrame(
    data=departments,
    columns=["department_id", "name"])

In [94]:
# save to disk into csv's
empdf.to_csv("employees.csv", index=False)
deptdf.to_csv("departments.csv", index=False)

In [93]:
# # close communications with db
cur.close()
conn.close()

## queries for creating neo4j graph

first you need to copy the `employees.csv` and `departments.csv` to /var/lib/neo4j/import

In [ ]:
CREATE CONSTRAINT employeeIdConstraint FOR (employee:Employee) REQUIRE employee.id IS UNIQUE

CREATE CONSTRAINT deptIdConstraint FOR (department:Department) REQUIRE department.id IS UNIQUE

// Import employees
LOAD CSV WITH HEADERS FROM "file:///employees.csv" AS row
CREATE (:Employee {
  `Employee ID`: toInteger(row.employee_id),
  full_name: row.full_name,
  Role: row.job_title
});

// Import departments
LOAD CSV WITH HEADERS FROM "file:///departments.csv" AS row
CREATE (:Department {
  `Department ID`: toInteger(row.department_id),
  name: row.name
});

// Create 'REPORTS_TO' relationships
LOAD CSV WITH HEADERS FROM "file:///employees.csv" AS row
WITH row
WHERE row.manager_id IS NOT NULL
MATCH (employee:Employee {employee_id: toInteger(row.employee_id)}),
      (manager:Employee {employee_id: toInteger(row.manager_id)})
CREATE (employee)-[:REPORTS_TO]->(manager);

// Create 'BELONGS_TO' relationships
LOAD CSV WITH HEADERS FROM "file:///employees.csv" AS row
WITH row
MATCH (employee:Employee {employee_id: toInteger(row.employee_id)}),
      (department:Department {department_id: toInteger(row.department_id)})
CREATE (employee)-[:BELONGS_TO]->(department);



## tests/experiments

In [ ]:
sudo neo4j-admin database import full --nodes=problem1/employees.csv --nodes=problem1/departments.csv --nodes=problem1/relationship.csv neo4j --overwrite-destination # --relation flag for relationships

CREATE CONSTRAINT employeeIdConstraint FOR (employee:Employee) REQUIRE employee.id IS UNIQUE

CREATE CONSTRAINT deptIdConstraint FOR (dept:Dept) REQUIRE dept.id IS UNIQUE
# CREATE INDEX FOR (c:Dept) ON (c.name)

LOAD CSV WITH HEADERS FROM "file:///employees.csv" AS csvLine
CREATE (employee:Employee {employee_id: toInteger(csvLine.employee_id), full_name: csvLine.full_name, Role: csvLine.job_title})
# CREATE (employee)-[:BELONGS_TO]->(dept)

LOAD CSV WITH HEADERS FROM "file:///departments.csv" AS csvLine
MERGE (department:Department {name: csvLine.name})
CREATE (dept:Dept {department_id: toInteger(csvLine.department_id), name: csvLine.name})

LOAD CSV WITH HEADERS FROM "file:///employees.csv"  AS row
WITH row
MATCH (employee:Employee {employee_id: toInteger(row.employee_id)}),
      (manager:Employee {employee_id: toInteger(row.manager_id)})
CREATE (employee)-[:REPORTS_TO]->(manager);

LOAD CSV WITH HEADERS FROM "file:///employees.csv" AS row
WITH row
MATCH
(department:Department {department_id: toInteger(row.department_id)}), (employee:Employee {employee_id: toInteger(row.employee_id)})
CREATE (employee)-[:BELONGS_TO]->(department);

:auto LOAD CSV WITH HEADERS FROM 'file:///relationship.csv' AS csvLine
CALL {
 WITH csvLine
 MATCH (employee:Employee {id: toInteger(csvLine.employeeId)}), (dept:Dept {id: toInteger(csvLine.departmentId)})
 MATCH (employee:Employee {id: toInteger(csvLine.employeeId)}),(manager:Manager {id: toInteger(csvLine.managerId)}) 
CREATE (employee)-[:BELONGS]->(dept)
CREATE (employee)-[:REPORTS]->(manager)
} IN TRANSACTIONS OF 2 ROWS


In [ ]:
# instead of manually creating below file use join

In [1]:
%%writefile relationship.csv
employeeId, departmentId, managerId
1,1,1
2,1,1
3,2,2
4,2,2
5,2,3
6,5,3
7,1,4
8,7,4

Writing relationship.csv
